In [1]:
def parse_range_string(range_string):
    """ 
    Args:
    - range_string (str): A string containing numbers and ranges, e.g., "1,3,5-7,9-11"
    
    Returns:
    - numbers (list of int): A list of numbers extracted from the range string.[1,3,5,6,7,9,10,11]
    """
    numbers = []
    
    # Split the string by commas to handle each part
    parts = range_string.split(',')
    
    for part in parts:
        if '-' in part:
            # If the part contains a hyphen, it's a range
            start, end = part.split('-')
            start, end = int(start), int(end)
            numbers.extend(range(start, end + 1))  # Use range to generate the numbers in the range
        else:
            # If the part is a single number, just add it to the list
            numbers.append(int(part))
    
    return numbers


In [2]:
import PIL
print('PIL',PIL.__version__)

PIL 10.2.0


In [3]:
import numpy
print('numpy',numpy.__version__)

numpy 1.26.3


In [4]:
import os
from PIL import Image
import torchvision.transforms as transforms

def img_to_tensor(image_path):

    # Args:
    # - image_path (str): Path to the image file, e.g., 'home/data/pic.jpg'

    # Returns:
    # - image_tensor (torch.Tensor): The image as a tensor.

    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    image_tensor = transform(image)
    return image_tensor

def load_images_to_tensor_list(folder_path,persons_range,person_img_range):

    persons_list=parse_range_string(persons_range)
    person_img_list=parse_range_string(person_img_range)
    
    # Args:
    # - folder_path (str): Path to the folder containing the images.
    
    # Returns:
    # - tensor_list (list of list of torch.Tensor): A 50x14 list of image tensors.
    
    tensor_list = []

    for i in persons_list:
        row = []
        for j in person_img_list:
            image_name = f"{i}-{j:02d}.jpg"  # Format image name jpg or jpeg or png
            image_path = os.path.join(folder_path, image_name)
            if os.path.exists(image_path):
                image_tensor = img_to_tensor(image_path)
                row.append(image_tensor)
            else:
                print(f"Image not found: {image_path}")
                break
        tensor_list.append(row)
    
    return tensor_list




person_matrix is a p x n matrix where each row represent  a different person. and elements of that row is tensor form of different image of that person.
each element of this matrix should be a  torch (3 x i_1 x i_2) tensor

In [5]:
import torch
def train(person_matrix,LS):

    p=len(person_matrix)
    n=len(person_matrix[0])
    sample_img=person_matrix[0][0]
    i_1=sample_img.size()[1]   # i_1
    i_2=sample_img.size()[2]   # i_2   

    # create a 3*i1*i2 tensor of zeros
    meanAll=torch.zeros(3,i_1,i_2)     # in torch (3*i1*i2) is same as (i1*i2*3) i paper
    for i in range(p):
        for j in range(n):
            meanAll=meanAll+person_matrix[i][j]
    meanAll=meanAll/(p*n)

    TensorCov=torch.zeros(3,i_2,i_2)
    for i in range(p):
        for j in range(n):
            temp=person_matrix[i][j]-meanAll
            TensorCov=TensorCov+torch.matmul(temp.mT, temp)
    TensorCov=TensorCov/(p*n)


    TenU,TenS,TenV_transpose=torch.linalg.svd(TensorCov, full_matrices=True) # here TenS is only the diagonal part


    TenFeature=[]
    for i in range(p):
        TenFeature.append([])
        for j in range(n):
            temp=torch.matmul(person_matrix[i][j], TenU[:,:,:LS])
            TenFeature[i].append(temp)

    return TenFeature, TenU[:,:,:LS]  # we need to pass this two as 2nd and 3rd parameter accordingly in eval function

In [6]:
def eval(target_img_tensor,TenFeature, TenU_LS):
    TenFeature_j=torch.matmul(target_img_tensor, TenU_LS)
    p=len(TenFeature)
    n=len(TenFeature[0])
    eval=torch.zeros(p,n)
    for i in range(p):
        for j in range(n):
            eval[i][j]=torch.norm(TenFeature[i][j]-TenFeature_j)
    # find p of the min value in eval
    min_val=eval.min()
    for i in range(len(eval)):
        for j in range(len(eval[0])):
            if eval[i][j] == min_val:
                return i+1
# we assome person class start from zero. so if 5 defferent person is there then the person classes (p) will be 0,1,2,3,4

In [7]:
# Define the path to the folder containing the images
folder_path = "/home/arya/my_python/my_torch/img"

# Load the images into a 50x14 list of tensors
image_tensor_train= load_images_to_tensor_list(folder_path,"1-50","1,3,5,7,9,11,13")

# Check if the list was created successfully
print(f"Successfully created a {len(image_tensor_train)}x{len(image_tensor_train[0])} list of image tensors.")

a=train(image_tensor_train,LS=14)

Successfully created a 50x7 list of image tensors.


In [8]:
test_person_tensor=load_images_to_tensor_list(folder_path,"1-50","2,4,6,8,10,12,14")
count=0
for i in range(50):
    for j in range (7):
        test_person_tensor[i][j]=eval(test_person_tensor[i][j],a[0],a[1])
print(test_person_tensor)

[[1, 1, 1, 1, 1, 1, 37], [2, 2, 2, 2, 2, 2, 37], [3, 3, 3, 3, 3, 3, 37], [4, 4, 4, 4, 4, 4, 37], [5, 5, 5, 5, 5, 5, 37], [6, 6, 6, 6, 19, 6, 37], [7, 7, 7, 7, 10, 7, 37], [8, 8, 8, 8, 8, 8, 37], [9, 9, 9, 9, 9, 9, 9], [10, 10, 10, 10, 10, 10, 10], [11, 11, 11, 11, 21, 11, 11], [12, 12, 12, 12, 25, 12, 12], [13, 13, 13, 13, 13, 13, 37], [14, 14, 14, 14, 13, 14, 37], [15, 15, 15, 15, 15, 15, 15], [16, 16, 16, 16, 16, 16, 16], [17, 17, 17, 17, 36, 17, 17], [18, 18, 18, 18, 18, 18, 20], [19, 19, 19, 19, 19, 19, 19], [20, 20, 20, 20, 18, 20, 20], [21, 21, 21, 21, 21, 21, 37], [22, 22, 22, 22, 8, 22, 37], [23, 23, 23, 23, 23, 23, 37], [24, 24, 24, 24, 24, 24, 37], [25, 25, 20, 25, 25, 25, 38], [26, 26, 26, 26, 26, 26, 38], [27, 27, 27, 27, 27, 27, 37], [28, 28, 11, 28, 28, 28, 37], [29, 29, 29, 29, 29, 29, 29], [30, 30, 30, 30, 30, 30, 30], [31, 31, 31, 31, 31, 31, 31], [32, 32, 32, 32, 32, 32, 32], [33, 33, 33, 33, 18, 33, 8], [34, 34, 34, 34, 34, 34, 37], [35, 35, 35, 35, 35, 35, 7], [36, 

In [9]:
def LS_accuracy_check(LS):
    # Define the path to the folder containing the images
    folder_path = "/home/arya/my_python/my_torch/img"

    # Load the images into a 50x14 list of tensors
    train_person_tensors= load_images_to_tensor_list(folder_path,"1-50","1,3,5,7,9,11,12,14")
    
    a=train(train_person_tensors,LS)

    test_person_tensor=load_images_to_tensor_list(folder_path,"1-50","2,4,6,8,10,13")
    count=0
    for i in range(50):
        for j in range (6):
            if i+1==eval(test_person_tensor[i][j],a[0],a[1]):
                count+=1
    accuracy=count*100/(50*6)
    print(f"for ls={LS} accuracy on test set is {accuracy}%")
    

In [10]:
LS_accuracy_check(1)
LS_accuracy_check(2)
LS_accuracy_check(3)
LS_accuracy_check(4) # till 4 it is good then start to get over fitted
LS_accuracy_check(5)
LS_accuracy_check(6)
LS_accuracy_check(7)
LS_accuracy_check(8)
LS_accuracy_check(9)
LS_accuracy_check(10)




for ls=1 accuracy on test set is 89.66666666666667%
for ls=2 accuracy on test set is 92.0%
for ls=3 accuracy on test set is 93.0%
for ls=4 accuracy on test set is 94.33333333333333%
for ls=5 accuracy on test set is 92.66666666666667%
for ls=6 accuracy on test set is 92.33333333333333%
for ls=7 accuracy on test set is 92.0%
for ls=8 accuracy on test set is 92.33333333333333%
for ls=9 accuracy on test set is 92.66666666666667%
for ls=10 accuracy on test set is 92.0%
for ls=11 accuracy on test set is 91.33333333333333%
for ls=12 accuracy on test set is 91.33333333333333%
for ls=13 accuracy on test set is 91.33333333333333%
for ls=14 accuracy on test set is 91.33333333333333%
